## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-24-03-42-38 +0000,bbc,Australia journalist unfairly fired over Gaza ...,https://www.bbc.com/news/articles/cly6dlpd332o...
1,2025-09-24-03-40-30 +0000,nyt,Drone Strike in Haiti Kills 8 Children at a Bi...,https://www.nytimes.com/2025/09/23/us/haiti-dr...
2,2025-09-24-03-39-00 +0000,wsj,Florida Gov. DeSantis Proposes Giving Trump Mi...,https://www.wsj.com/us-news/florida-gov-desant...
3,2025-09-24-03-38-49 +0000,startribune,Takeaways: Lynx completely fall apart in Game ...,https://www.startribune.com/minnesota-lynx-pho...
4,2025-09-24-03-35-45 +0000,bbc,High prices hit sales of popular blooms - florist,https://www.bbc.com/news/articles/cj4y18kqwgdo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2292/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,75
53,kimmel,17
121,jimmy,13
234,was,12
580,un,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
29,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/the-franti...,149
140,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...,130
161,2025-09-23-18-26-27 +0000,nypost,Trump supports NATO countries shooting down Ru...,https://nypost.com/2025/09/23/us-news/trump-su...,123
264,2025-09-23-12-21-27 +0000,nypost,Trump to tout US as defender of Western civili...,https://nypost.com/2025/09/23/us-news/trump-to...,120
310,2025-09-23-08-03-21 +0000,nypost,Ukraine seeks support as Zelensky visits US to...,https://nypost.com/2025/09/23/world-news/ukrai...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
29,149,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/the-franti...
219,69,2025-09-23-15-26-23 +0000,latimes,Nexstar and Sinclair TV stations will not run ...,https://www.latimes.com/entertainment-arts/bus...
171,55,2025-09-23-17-53-00 +0000,wsj,Fed Chair Jerome Powell said he judged the Fed...,https://www.wsj.com/economy/central-banking/fe...
140,51,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...
199,44,2025-09-23-16-52-00 +0000,wsj,The Secret Service said it dismantled a networ...,https://www.wsj.com/politics/national-security...
136,41,2025-09-23-19-52-00 +0000,wsj,"Canada Needs to Chart New Economic Course, Ban...",https://www.wsj.com/articles/canada-needs-to-c...
184,41,2025-09-23-17-29-24 +0000,nypost,Megyn Kelly slams ‘coward’ Van Jones for brand...,https://nypost.com/2025/09/23/media/megyn-kell...
161,37,2025-09-23-18-26-27 +0000,nypost,Trump supports NATO countries shooting down Ru...,https://nypost.com/2025/09/23/us-news/trump-su...
18,36,2025-09-24-02-36-48 +0000,nypost,24-year-old illegal immigrant who posed as Ohi...,https://nypost.com/2025/09/23/us-news/24-year-...
322,31,2025-09-23-05-31-13 +0000,bbc,Copenhagen and Oslo airports forced to close t...,https://www.bbc.com/news/articles/cn4lj1yvgvgo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
